In [1]:
import pandas as pd
import numpy as np

from src.PortfolioIndicator import PortfolioIndicator
from src.PortfolioConstructor import PortfolioConstructor

from linearmodels import PanelOLS,FamaMacBeth

In [2]:
import statsmodels.api as sm

In [3]:
df = pd.read_csv("data/yahoo_finance_data.csv")
df["date"] = pd.to_datetime(df["date"])

In [4]:
vol_df = df.pivot_table(columns="ticker",values="volume",index="date")
price_df = df.pivot_table(columns="ticker",values="price",index="date")

In [10]:
universe_df = price_df

In [11]:
momentum_period = 1
waiting_time = 1

In [12]:
momentum = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
momentum = momentum.stack()

In [26]:
vol_control = vol_df
vol_control = vol_control.stack()

In [27]:
vol_control

date        ticker  
2017-12-31  ADA-USD     305947008.0
            ADX-USD      26397700.0
            AE-USD        1482490.0
            ANT-USD       1516050.0
            ARDR-USD     35154900.0
                           ...     
2021-01-03  XVG-USD      23389208.0
            XZC-USD       3361756.0
            ZEC-USD     723282432.0
            ZEN-USD      20377882.0
            ZRX-USD      71434048.0
Length: 112690, dtype: float64

In [28]:
future_return_days = 1

future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))
future_return_df_long = future_return_df.stack()

future_return_df_long = pd.DataFrame(future_return_df_long)

In [41]:
future_return_df_long.columns = ["ret"]

In [42]:
signal_df = pd.concat([momentum,
                       vol_control],axis=1)

In [43]:
signal_df.columns = ["momentum","vol"]

In [59]:
merge_df = future_return_df_long.merge(signal_df,left_index=True,right_index=True)

In [60]:
merge_df = merge_df.dropna()

In [61]:
index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]


index_df["ret"] = merge_df["ret"]
index_df["momentum"] = merge_df["momentum"]
index_df["vol"] = merge_df["vol"]

index_df.index = range(0,index_df.shape[0])

merge_df = index_df

merge_df = merge_df.set_index(["asset","date"])
merge_df = merge_df.dropna()

In [62]:
exog_vars = ["momentum","vol"]
depe_var = ["ret"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [63]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,ret,R-squared:,0.0015
Estimator:,PanelOLS,R-squared (Between):,-0.0789
No. Observations:,112484,R-squared (Within):,0.0015
Date:,"Sat, Jan 23 2021",R-squared (Overall):,0.0014
Time:,12:41:24,Log-likelihood,3.411e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,81.916
Entities:,103,P-value,0.0000
Avg Obs:,1092.1,Distribution:,"F(2,111285)"
Min Obs:,951.00,,
Max Obs:,1095.0,F-statistic (robust):,6.7337


In [64]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

Dep. Variable:,ret,R-squared:,0.0014
Estimator:,PanelOLS,R-squared (Between):,-0.0802
No. Observations:,112484,R-squared (Within):,0.0015
Date:,"Sat, Jan 23 2021",R-squared (Overall):,0.0014
Time:,12:42:28,Log-likelihood,3.023e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,79.199
Entities:,103,P-value,0.0000
Avg Obs:,1092.1,Distribution:,"F(2,112481)"
Min Obs:,951.00,,
Max Obs:,1095.0,F-statistic (robust):,5.4445


In [66]:
mod = FamaMacBeth(depe,exog)
res = mod.fit(cov_type='kernel',kernel="bartlett")

In [67]:
res

Dep. Variable:,ret,R-squared:,-0.0150
Estimator:,FamaMacBeth,R-squared (Between):,-0.4683
No. Observations:,112484,R-squared (Within):,-0.0146
Date:,"Sat, Jan 23 2021",R-squared (Overall):,-0.0150
Time:,12:43:02,Log-likelihood,2.932e+04
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,-833.00
Entities:,103,P-value,1.0000
Avg Obs:,1092.1,Distribution:,"F(2,112481)"
Min Obs:,951.00,,
Max Obs:,1095.0,F-statistic (robust):,13.655


### Add Bitcoin/Market Return

In [71]:
btc_price = price_df["BTC-USD"]
eth_price = price_df["ETH-USD"]

In [76]:
btc_ret = btc_price.pct_change()
eth_ret = eth_price.pct_change()

In [91]:
market_ret = pd.concat([btc_ret,eth_ret],axis=1).dropna()

In [82]:
ret_df = price_df.pct_change().dropna()

In [98]:
y = ret_df[["ADA-USD"]]
x = market_ret
x = sm.add_constant(x)

In [96]:
from statsmodels.regression.linear_model import OLS

In [99]:
mod = OLS(y,x)

In [102]:
res = mod.fit()

In [106]:
btc_beta = res.params["BTC-USD"]
eth_beta = res.params["BTC-USD"]

0.2575965779210107

In [127]:
def calculate_beta_series(y,x,length=20):
    
    btc_list = []
    eth_list = []
    date_list = []
    
    for idx in range(length,y.shape[0]):
        
        x_sub = x[(idx-length):idx]
        y_sub = y[(idx-length):idx]
        
        mod = OLS(y_sub,x_sub)
        res = mod.fit()
        
        btc_beta = res.params["BTC-USD"]
        eth_beta = res.params["BTC-USD"]
        
        
        date = y_sub.index[-1]

        btc_list.append(btc_beta)
        eth_list.append(eth_beta)
        date_list.append(date)
    
    df = pd.DataFrame({"date":date_list,"btc_beta":btc_list,"eth_list":eth_list})
    
    return df

In [128]:
calculate_beta_series(y,x,length=20)

,date,btc_beta,eth_list
0,2018-01-20,0.424359,0.424359
1,2018-01-21,0.445380,0.445380
2,2018-01-22,0.485425,0.485425
3,2018-01-23,0.577769,0.577769
4,2018-01-24,0.586116,0.586116
...,...,...,...
1071,2020-12-29,-0.014780,-0.014780
1072,2020-12-30,-0.170332,-0.170332
1073,2020-12-31,-0.160128,-0.160128
1074,2021-01-01,-0.159600,-0.159600
